In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imread
import cv2

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from glob import glob
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# ../input/
PATH = os.path.abspath(os.path.join('..', 'input'))

# ../input/sample/images/
SOURCE_IMAGES = os.path.join(PATH, "sample", "images")

# ../input/sample/images/*.png
images = glob(os.path.join(SOURCE_IMAGES, "*.png"))

# Load labels
labels = pd.read_csv('../input/sample_labels.csv')

In [ ]:
labels.head(5)

In [ ]:
# load in all of the images
labels['path'] = labels['Image Index'].apply(lambda x: os.path.join(SOURCE_IMAGES,x))
labels['image'] = labels['path'].map(imread)

In [ ]:
# see the image size distribution
labels['image'].map(lambda x: x.shape).value_counts()

In [ ]:
labels['image'][0].shape

In [ ]:
labels['NumLabel']=labels['Finding Labels'].apply(lambda x: 0 if 'No Finding' in x else 1)

In [ ]:
IMG_SIZE = (128,128)
print(IMG_SIZE[0])
def get_data():
    imgs_x = []
    label_y = []
    for index, row in labels.iterrows():
        if row['image'].shape == (1024,1024):
            imgs_x.append(cv2.resize(row['image'], (IMG_SIZE[0],IMG_SIZE[1]), interpolation=cv2.INTER_CUBIC).flatten())
            label_y.append(row['NumLabel'])
    return imgs_x, label_y    

In [ ]:
X, Y = get_data()
X = np.array(X)/255
#X= np.reshape(X, X.shape + (1,))
Y = np.array(Y)

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split    
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2018)
print('Train Shape', X_train.shape, 'test shape', X_test.shape)

##flatten the data into 1-D
X_train = X_train.flatten()
X_test = X_test.flatten()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
##K-NN
from sklearn.neighbors import KNeighborsClassifier
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = KNeighborsClassifier(n_neighbors=1000)
model.fit(X_train, y_train)
acc = model.score(X_test, y_test)
print("[INFO] k-NN classifier: k=%d" % 1000)
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))

In [ ]:
#neural network
from sklearn.neural_network import MLPClassifier
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = MLPClassifier(hidden_layer_sizes=(5,), max_iter=1000, alpha=1e-4,
                      solver='sgd', tol=1e-4, random_state=1,
                      learning_rate_init=1e-3)
model.fit(X_train, y_train)
acc = model.score(X_test, y_test)
print("[INFO] hidden_layer_size is: %d" % 5)
print("[INFO] neural network raw pixel accuracy: {:.2f}%".format(acc * 100))

In [ ]:
#SVC
from sklearn.svm import SVC
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = SVC(max_iter=2000,class_weight='balanced',C = 5, gamma = 0.1)
model.fit(X_train, y_train)
acc = model.score(X_test, y_test)
print("[INFO] SVM-SVC raw pixel accuracy: {:.2f}%".format(acc * 100))

#from sklearn.grid_search import GridSearchCV
#param_grid = {'C':[0.1,1,10,100],'gamma':[1,0.1,0.01,0.001]}
#grid = GridSearchCV(SVC(),param_grid,verbose=1)
#grid.fit(X_train, y_train)

In [ ]:
##Random Forest
from sklearn.ensemble import RandomForestClassifier
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = RandomForestClassifier(n_estimators=200)
model.fit(X_train, y_train)
acc = model.score(X_test, y_test)
print("[INFO] Random Forest raw pixel accuracy: {:.2f}%".format(acc * 100))
y_predict = model.predict(X_test)
print(confusion_matrix(y_test, y_predict))
print('\n')
print(classification_report(y_test,y_predict))